In [1]:
from collections import deque 

class flipflop:
    def __init__(self, destinations):
        self.state = False
        self.destinations = destinations
    def pulse(self, p, origin):
        # Origin is unused
        if(p == "high"):
            return None
        else: # p == "low"
            self.state = not self.state
            if(self.state == True):
                return "high"
            else:
                return "low"
    def get_state(self):
        return int(self.state)
    def add_input(self, input):
        pass
    def get_destinations(self):
        return self.destinations

class conjunction:
    def __init__(self, destinations):
        self.inputs = {}
        self.destinations = destinations
    def pulse(self, p, origin):
        self.inputs[origin] = p
        # print("Checking inputs", self.inputs)
        return self.get_hl()
    def get_hl(self):
        if("low" in self.inputs.values()):
            return "high"
        else:
            return "low"
    def get_state(self):
        return 1 if self.state == 'high' else 0
    def add_input(self, input):
        self.inputs[input] = "low"
    def get_destinations(self):
        return self.destinations

def pushButton(modules, oldstart, keysToWatch = []):
    start = oldstart.copy()
    out = {"high":0, "low":1}

    # print("Sent from button to broadcaster with low")
    # for s in start:
        # print("Sent from broadcaster to ", s[0], " with ", s[1])
    while len(start):
        (key, value, origin) = start.popleft()
        out[value] += 1
        if(key in modules):
            m = modules[key]
            newPulse = m.pulse(value, origin)
            # print("checking key in keystowatch", key, keysToWatch)
            if key in keysToWatch:
                # print("found a key I care about", key)
                if newPulse == 'high':
                    # print("found a high pulse I care about", key)
                    out[key] = value

            if(newPulse != None):
                for d in m.get_destinations():
                    start.append((d, newPulse, key))
        else:
            # print("Error: ", key, " not in modules", value)
            if key not in out.keys():
               out[key] = value
                # print("Sending from ", key, " to ", d, " with ", newPulse)
    return out


# Part 1

In [2]:

lines = open('20.txt').read().splitlines()

modules = {}
start = deque()
for line in lines:
    key, destination = line.split(" -> ")
    dests = destination.split(", ")
    keyType = key[0]
    keyTag = key[1:]
    # print(keyType, keyTag, destination)
    if keyType == 'b':
       
        for dest in dests:
            start.append((dest, "low", "broadcast"))
    elif keyType == '%':
        m = flipflop(dests)
        modules[keyTag] = m
    elif keyType == '&':
        m = conjunction(dests)
        modules[keyTag] = m
for m in modules.keys():
    for d in modules[m].get_destinations():
        if(d in modules.keys()):
            # print("Adding ", m , " to ", d  )
            modules[d].add_input(m)



In [3]:

outs = {"high":0, "low":0}
for i in range(1000):
    # print("-------")
    out = pushButton(modules, start)
    # print(out)
    for k in out.keys():
        if k in outs.keys(): outs[k] += out[k]
    # if('rx' in out.keys()):
        # print(i, "RX", out['rx'])

print(outs['high'] * outs['low'])

807069600


# Part 2

In [4]:
lines = open('20.txt').read().splitlines()

modules = {}
start = deque()
flipflops = []

for line in lines:
    key, destination = line.split(" -> ")
    dests = destination.split(", ")
    keyType = key[0]
    keyTag = key[1:]
    # print(keyType, keyTag, destination)
    if keyType == 'b':
       
        for dest in dests:
            start.append((dest, "low", "broadcast"))
    elif keyType == '%':
        m = flipflop(dests)
        flipflops.append(m)
        modules[keyTag] = m
    elif keyType == '&':
        m = conjunction(dests)
        modules[keyTag] = m
for m in modules.keys():
    for d in modules[m].get_destinations():
        if(d in modules.keys()):
            # print("Adding ", m , " to ", d  )
            modules[d].add_input(m)

finalInput = [k for k in modules.keys() if "rx" in modules[k].get_destinations()].pop()
previousInputs = list(modules[finalInput].inputs.keys())

# previousInputs

In [5]:
import math 
values = []
spaces = {}
for k in previousInputs:
    spaces[k] = []
for i in range(10000):
    out= pushButton(modules, start, previousInputs)
    for k in previousInputs:
        if k in out.keys():
            spaces[k].append(i)

for k in spaces:
    v = spaces[k]
    start = 0

lasts = []
for k in spaces:
    v = spaces[k]
    lasts.append(v[-1] - v[-2])

lcm = lasts[0];
for i in lasts[1:]:
  lcm = lcm*i//math.gcd(lcm, i)

print(lcm)

221453937522197
